<a href="https://colab.research.google.com/github/alexisrisaldi/Data-deparment/blob/master/Ciudades_WPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install reverse_geocode
!pip install XlsxWriter

In [ ]:
import pandas as pd
import reverse_geocode
from shapely.geometry import Point
import geopandas as gpd

In [ ]:
# Leer una hoja específica llamada "Familias"
df_families = pd.read_excel('Culture LightZ Survey.xlsx', sheet_name='Families')

# Leer una hoja específica llamada "Members"
df_members = pd.read_excel('Culture LightZ Survey.xlsx', sheet_name='Members')

# Leer una hoja específica llamada "Indicators"
df_indicators = pd.read_excel('Culture LightZ Survey.xlsx', sheet_name='Indicators')

# Leer una hoja específica llamada "Priorities"
df_priorities = pd.read_excel('Culture LightZ Survey.xlsx', sheet_name='Priorities')

# Leer una hoja específica llamada "Priorities"
df_achievements = pd.read_excel('Culture LightZ Survey.xlsx', sheet_name='Achievements')

# Leer una hoja específica llamada "Notes"
df_notes = pd.read_excel('Culture LightZ Survey.xlsx', sheet_name='Notes')

# Leer una hoja específica llamada "Interventions"
df_interventions = pd.read_excel('Culture LightZ Survey.xlsx', sheet_name='Interventions')

In [ ]:
df_families.count()

In [ ]:
#ordeno los campos de la hoja original
df_families.sort_values(by=['Family code', 'Survey number'], inplace=True)

In [ ]:
# crea un nuevo DF en base a algunas columnas especificas
df_ciudades = df_families[['Family code', 'Survey number','Family location']]

In [ ]:
df_ciudades

In [ ]:
df_ciudades['Family code'].value_counts()

In [ ]:
# ordenar valores
df_ciudades.sort_values(by=['Family code', 'Survey number'], inplace=True)

<ipython-input-96-f4314ce2be59>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ciudades.sort_values(by=['Family code', 'Survey number'], inplace=True)


In [ ]:
df_ciudades

In [ ]:
# verificar si hay nulos
df_ciudades['Family location'].isnull().sum()


np.int64(0)

In [ ]:
# Separar la columna 'Family location' en dos nuevas columnas
df_ciudades[['Latitude', 'Longitude']] = df_ciudades['Family location'].str.split(';', expand=True)

# Convertir las nuevas columnas a tipo numérico (float)
df_ciudades['Latitude'] = df_ciudades['Latitude'].astype(float)
df_ciudades['Longitude'] = df_ciudades['Longitude'].astype(float)


In [ ]:
df_ciudades

In [ ]:
#Funciones para convertir latitud y longitud en una nueva columna de ciudad y país (Asumiendo que el formato de las coordenadas sea correcto)
def get_city(row):
    coordinates = (row['Latitude'], row['Longitude'])
    result = reverse_geocode.get(coordinates)
    return result['city']

def get_country(row):
    coordinates = (row['Latitude'], row['Longitude'])
    result = reverse_geocode.get(coordinates)
    return result['country']

In [ ]:
df_ciudades['Country_Ver'] = df_ciudades.apply(get_country, axis=1)
df_ciudades['City'] = df_ciudades.apply(get_city, axis=1)

In [ ]:
#imprimir para ver si salio bien
df_ciudades

In [ ]:
df_city = df_ciudades[['Family code', 'Survey number','City','Country_Ver']]

In [ ]:
df_city

In [ ]:
df_resultado_families = pd.merge(
    df_families,
    df_city,
    on=['Family code', 'Survey number'],
    how='inner'   # Tipo de join: inner, left, right, outer
)

In [ ]:
df_resultado_families

In [ ]:
df_resultado_families.drop(columns=['Family location'], inplace=True)


In [ ]:
df_resultado_families

In [ ]:
# Crear el archivo Excel con todas las hojas
with pd.ExcelWriter('Culture LightZ Survey_New.xlsx', engine='xlsxwriter') as writer:
    df_resultado_families.to_excel(writer, sheet_name='Families', index=False)
    df_members.to_excel(writer, sheet_name='Members', index=False)
    df_indicators.to_excel(writer, sheet_name='Indicators', index=False)
    df_priorities.to_excel(writer, sheet_name='Priorities', index=False)
    df_notes.to_excel(writer, sheet_name='Notes', index=False)
    df_interventions.to_excel(writer, sheet_name='Interventions', index=False)
    df_achievements.to_excel(writer, sheet_name='Achievements', index=False)

In [ ]:
# Descargar el archivo en tu PC
from google.colab import files
files.download('Aspire(Spanish)_new.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>